<a href="https://colab.research.google.com/github/kurtlee1984/ML_20210602/blob/main/poem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from urllib.request import urlretrieve
url = "https://raw.githubusercontent.com/Elwing-Chou/ml0602/main/poem_train.csv"
urlretrieve(url, "train.csv")
url = "https://raw.githubusercontent.com/Elwing-Chou/ml0602/main/poem_test.csv"
urlretrieve(url, "test.csv")

('test.csv', <http.client.HTTPMessage at 0x7f6683fcbe90>)

In [31]:
import pandas as pd
train = pd.read_csv("train.csv", encoding="utf-8")
test = pd.read_csv("test.csv", encoding="utf-8")
# train
# test

In [36]:
# 分類: 輸出 0 1 2 3
# series.replace(字典)
# {"李白":0, "杜甫":1, "白居易":2}
# series.unique/series.value_counts
u = train["作者"].unique()
# u = train["作者"].value_counts()
# print(u)
name2index = {n:i for i, n in enumerate(u)}
index2name = {i:n for i, n in enumerate(u)}
# print(name2index)
# print(index2name)
# y_train = train["作者"].replace(name2index)
# y_test = test["作者"].replace(name2index)
# y_test

{'李白': 0, '杜甫': 1, '白居易': 2}


In [ ]:
import jieba
def poemcut(p):
  return " ".join(jieba.cut(p))
x_train = train["內容"].apply(poemcut)
x_test = test["內容"].apply(poemcut)
x_train

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
vec.fit(x_train)
len(vec.vocabulary_)

In [ ]:
#稀疏矩陣
x_train_count = vec.transform(x_train)
x_test_count = vec.transform(x_test)
x_test_count

In [20]:
# 檢查標點符號, \r \n
# vec.vocabulary_["。"]
# vec.vocabulary_["\r\n"]

In [18]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha=0.01)
clf.fit(x_train_count, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [19]:
from sklearn.metrics import accuracy_score
pre = clf.predict(x_test_count)
accuracy_score(pre, y_test)

0.8

In [27]:
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(y_test, pre)
pd.DataFrame(mat,
            columns=["{}(預測)".format(n) for n in u],
            index=["{}(正確)".format(n) for n in u])

,李白(預測),杜甫(預測),白居易(預測)
李白(正確),8,1,1
杜甫(正確),1,8,1
白居易(正確),1,1,8


In [38]:
poem = input("輸入詩:")
poems = [poemcut(poem)]
poems_count = vec.transform(poems)
pre = clf.predict(poems_count)[0]
print("答案是:", index2name[pre])
pre_proba = clf.predict_proba(poems_count)[0]
for name, proba in zip(u, pre_proba):
    print(name, "的機率:", proba)

輸入詩:金樽清酒斗十千
答案是: 李白
李白 的機率: 0.7802293304852961
杜甫 的機率: 0.09090812069152791
白居易 的機率: 0.12886254882317483
